In [ ]:
# Instalar librerías en las versiones utilizadas en este trabajo
!pip install -r requirements.txt

## Introducción
En este código analizamos los datos provenientes del scrapeo para 3 ambientes.

Importamos librerías y bases a utilizar.

In [20]:
# Importamos librerías
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
import os

In [21]:
# Importar archivos
ruta_base = os.path.join(os.pardir, "Bases")
df_argenprop = pd.read_excel(ruta_base + "/Bases scrapeo/scraping_3amb_2024-12-14.xlsx")

In [22]:
#Separamos la moneda (USD o $) del valor del precio en dos columnas
df_temp = df_argenprop['Precio'].str.split(' ', expand = True)
df_argenprop['Moneda'] = df_temp[0]
df_argenprop['Precio'] = df_temp[1]


In [ ]:
df_argenprop

# Limpieza y análisis

In [ ]:
# Sacamos los puntos para que el precio sea solo número
df_argenprop['Precio'] = df_argenprop['Precio'].str.replace(".", "")
# Convertimos la columna precio a valores numéricos
df_argenprop["Precio"] = pd.to_numeric(df_argenprop["Precio"], errors="coerce")
# eliminamos los casos vacíos
df_argenprop = df_argenprop.dropna()
# Convertimos la columna precio a int
df_argenprop["Precio"] = df_argenprop["Precio"].astype(int)

#Reseteamos el index
df_argenprop = df_argenprop.reset_index(drop=True)

In [ ]:
df_argenprop

In [ ]:
# Verificamos que precio haya quedado como integer
df_argenprop['Precio'].dtype

In [27]:
# Definimos comunas
COMUNA1 = ['Retiro', 'San Nicolás', 'Puerto Madero', 'San Telmo', 'Monserrat', "Montserrat", 'Constitución', "Tribunales", "Constitucion", "Barrio San Pedro", "Centro", "Microcentro"]
COMUNA2 = ['Recoleta', "Barrio Norte", "Plaza San Martín"]
COMUNA3 = ['Balvanera', 'San Cristóbal', "Abasto", "Congreso", "Once", "San Cristobal"]
COMUNA4  = ['La Boca', 'Barracas', 'Parque Patricios', 'Nueva Pompeya', "Boca", "Pompeya", "Catalinas"]
COMUNA5  = ['Almagro' , 'Boedo', "Almagro Norte", "Almagro Sur"]
COMUNA6  = ['Caballito', "Caballito Norte", "Caballito Sur", "Cid Campeador", "Parque Rivadavia", "Parque Centenario", "Primera Junta"]
COMUNA7  = ['Flores' , 'Parque Chacabuco', "Flores Norte", "Flores Sur"]
COMUNA8  = ['Villa Soldati', 'Villa Riachuelo' , 'Villa Lugano']
COMUNA9  = ['Liniers', 'Mataderos' , 'Parque Avellaneda']
COMUNA10 = ['Villa Real','Monte Castro','Versalles','Floresta','Vélez Sarfield', "Velez Sarsfield", 'Villa Luro', "Floresta Sur", "Floresta Norte"]
COMUNA11 = ['Villa General Mitre','Villa Devoto','Villa del Parque', 'Villa Santa Rita']
COMUNA12 = ['Coghlan','Saavedra','Villa Urquiza' , 'Villa Pueyrredón', "Villa Pueyrredon", "Urquiza R"]
COMUNA13 = ['Núñez', "Nuñez", 'Belgrano','Colegiales', "Belgrano Barrancas", "Belgrano C", "Belgrano R", "Belgrano Chico", "Lomas de Nuñez", "River"]
COMUNA14 = ['Palermo', "Botanico", "Parque Las Heras", "Las Cañitas", "Palermo Chico", "Palermo Hollywood", "Palermo Nuevo", "Palermo Soho", "Palermo Viejo", ]
COMUNA15 = ['Chacarita','Villa Crespo','La Paternal','Villa Ortúzar', "Villa Ortuzar", 'Agronomía' ,'Parque Chas', "Agronomia", "Paternal"]

In [28]:
# Definimos una función para asignar comuna al barrio
comunas = [COMUNA1, COMUNA2, COMUNA3, COMUNA4, COMUNA5, COMUNA6, COMUNA7, COMUNA8, COMUNA9, COMUNA10, COMUNA11, COMUNA12, COMUNA13, COMUNA14, COMUNA15,]
def barrio2comuna(barrio):
  for c in range(0,15):
    if barrio in comunas[c]:
      return(c+1)
  return('No se encuentra')

In [ ]:
#Aplicamos la función barrio2comuna solo a las filas de la columna Barrio
df_argenprop['Comuna'] = df_argenprop['Barrio'].apply(barrio2comuna)

print(df_argenprop)

In [30]:
# Usamos filtro para chequear que todos los barrios se identifiquen con una comuna y, en caso contrario, asignar el barrio a la comuna

filtro = df_argenprop['Comuna'] == 'No se encuentra'
df_filtrado_sincomunas = df_argenprop[filtro]

df_filtrado_sincomunas

df_filtrado_sincomunas.to_excel(ruta_base + '/Tablas y bases análisis scraping/argenprop_comunas_missing_3amb.xlsx', index=False)

In [ ]:
df_filtrado_sincomunas

In [32]:
#Filtramos los valores en pesos para deshacernos de valores con errores / outliers
filtro = (df_argenprop["Precio"] > 200000) & (df_argenprop["Precio"] < 5000000) & (df_argenprop["Moneda"] == '$')
df_filtrado_pesos = df_argenprop[filtro]

#Reseteamos el index
df_filtrado_pesos = df_filtrado_pesos.reset_index(drop=True)

In [ ]:
df_filtrado_pesos

In [ ]:
pd.set_option('display.float_format', '{:,.0f}'.format) # Le pedimos que no use notación científica
df_filtrado_pesos['Precio'].describe()

In [35]:
#Hacemos lo mismo con los valores en dólares
filtro = (df_argenprop["Precio"] > 100) & (df_argenprop['Precio'] < 5100) & (df_argenprop["Moneda"] == 'USD')
df_filtrado_dolares = df_argenprop[filtro]

#Reseteamos el index
df_filtrado_dolares = df_filtrado_dolares.reset_index(drop=True)

In [ ]:
df_filtrado_dolares

In [ ]:
df_filtrado_dolares['Precio'].describe()

In [38]:
#Importamos precios históricos del dólar blue por día
df_dolar = pd.read_excel(ruta_base + "/Bases auxiliares/dolar blue histórico.xlsx")

In [39]:
# Nos quedamos con las columnas de precio de venta y fecha
df_dolar = df_dolar.drop('Compra', axis = 1)

In [ ]:
#Verificamos que la columna Fecha esté seteada como datetime
df_dolar.dtypes

In [ ]:
# Verificamos lo mismo para la columna fecha del df_filtrado_dolares
df_filtrado_dolares.dtypes

In [42]:
# Hacemos un merge que permita asignar a cada fila el valor del dólar de acuerdo con la fecha en que fue scrapeado
df_dolar_a_pesos = pd.merge(df_filtrado_dolares, df_dolar, on='Fecha', how='left')

In [ ]:
df_dolar_a_pesos

In [ ]:
# Renombramos la columna del precio del dólar
df_dolar_a_pesos = df_dolar_a_pesos.rename(columns = {'Venta' : 'Precio dólar'})
df_dolar_a_pesos

In [45]:
# Pasamos el df en dólares a pesos al precio del dolar blue del día en que se realizó el scrapeo
df_dolar_a_pesos['Precio'] = df_dolar_a_pesos['Precio'] * df_dolar_a_pesos['Precio dólar']
df_dolar_a_pesos["Precio"] = pd.to_numeric(df_dolar_a_pesos["Precio"], errors="coerce")
df_dolar_a_pesos = df_dolar_a_pesos.drop('Precio dólar', axis = 1)
df_dolar_a_pesos['Moneda'] = '$'

In [ ]:
df_dolar_a_pesos

In [ ]:
# Creamos un df con todos los valores en pesos para poder trabajar con todos los casos y sea comparable
df_unido = pd.concat([df_dolar_a_pesos, df_filtrado_pesos], ignore_index = True)

df_unido

## Tablas resumen

In [ ]:
# Armamos tabla con medidas resumen del df unido
pd.set_option('display.float_format', '{:,.0f}'.format) # Le pedimos que no use notación científica
df_tablas_resumen = df_unido['Precio'].describe()

print(df_tablas_resumen)

df_tablas_resumen.to_excel(ruta_base + '/Tablas y bases análisis scraping/Tabla_resumen_caba_3amb.xlsx')

In [ ]:
# Armamos un groupby para sacar estadísticos por comuna
df_estadisticasxcomuna = df_unido.groupby("Comuna", as_index=False)["Precio"].describe()

print(df_estadisticasxcomuna)

df_estadisticasxcomuna.to_excel(ruta_base + '/Tablas y bases análisis scraping/Estadisticas_por_comuna_3amb.xlsx')

In [ ]:
# Armamos un groupby para sacar precio promedio por comuna
df_promedioxcomuna = df_unido.groupby("Comuna", as_index=False)["Precio"].mean()
df_promedioxcomuna['Precio'] = df_promedioxcomuna['Precio'].round(1) #Redondeamos a 1 decimal
# Renombramos la columna
df_promedioxcomuna.rename(columns={"Precio": "Precio promedio"}, inplace=True)

print(df_promedioxcomuna)


In [ ]:
# Armamos groupby para ver cantidad de publicaciones por comuna
df_publicaciones_xcomuna = df_unido.groupby("Comuna", as_index=False)['Precio'].count()
# Cambiamos nombre de la columna
df_publicaciones_xcomuna.rename(columns={"Precio": "Cantidad de publicaciones"}, inplace=True)

df_publicaciones_xcomuna

## Gráficos

In [52]:
# subimos archivo geográfico de comunas de caba
comunas = gpd.read_file(ruta_base + "/Archivos geográficos comunas/comunas.shp")

In [ ]:
comunas

In [54]:
# Mergeamos los datos geográficos de comunas con nuestro df de precio promedio por comuna con key = comuna
comunas_mapa_precio = comunas.merge(df_promedioxcomuna, left_on= 'comuna', right_on= 'Comuna')

In [ ]:
# Verificamos el merge
comunas_mapa_precio

In [ ]:
# Graficamos mapa precio promedio por comuna

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
comunas_mapa_precio.plot(
    column="Precio promedio",
    ax=ax,
    cmap="Blues",
    vmin = 500000,
    vmax = 1300000,
    edgecolor="black")

# Acá admitimos que le pedimos mucha ayuda a ChatGPT que nos armó un for para asignar las etiquetas a las comunas correspondientes

for idx, row in comunas_mapa_precio.iterrows():
    # Obtenemos el centroide de la geometría para colocar la etiqueta
    centroid = row.geometry.centroid
    # Agregamos la etiqueta en el centroide, usando el valor de la columna correspondiente
    ax.text(centroid.x, centroid.y,
            f'{row["Precio promedio"]}',
            fontsize=8,
            ha='center',
            color='black')

plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# Mergeamos los datos geográficos de comunas con nuestro df de cantidad de publicaciones por comuna con key = comuna
comunas_mapa_publicaciones = comunas.merge(df_publicaciones_xcomuna, left_on= 'comuna', right_on= 'Comuna')

comunas_mapa_publicaciones

In [ ]:
# Graficamos mapa cantidad de publicaciones por comuna
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
comunas_mapa_publicaciones.plot(column="Cantidad de publicaciones",
             ax=ax,
             cmap="Greens",
             edgecolor="black")

for idx, row in comunas_mapa_publicaciones.iterrows():
    centroid = row.geometry.centroid
    ax.text(centroid.x, centroid.y,
            f'{row["Cantidad de publicaciones"]}',
            fontsize=8,
            ha='center',
            color='black')

plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# Hacemos groupby para contar cantidad de publicaciones en dólares por comuna
df_publicaciones_dolar = df_filtrado_dolares.groupby("Comuna", as_index=False)['Precio'].count()
# Renombramos columna
df_publicaciones_dolar.rename(columns={"Precio": "Cantidad de publicaciones en dolares"}, inplace=True)

df_publicaciones_dolar

In [ ]:
# Mergeamos df de cantidad de publicaciones por comuna total con el de cantidad de publicaciones en dólares con key = comuna para poder hacer división despues
# usamos how = outer porque teníamos tres comunas sin publicaciones en dólares pero no queríamos perder el dato
df_prop_dolar = df_publicaciones_xcomuna.merge(df_publicaciones_dolar, left_on= 'Comuna', right_on= 'Comuna', how='outer')

df_prop_dolar

In [ ]:
# Sacamos proporción de publicaciones en dólares sobre publicaciones totales para cada comuna * 100 (para tenerlo en %)
df_prop_dolar['Proporcion de publicaciones en dolares'] = df_prop_dolar['Cantidad de publicaciones en dolares'] / df_prop_dolar['Cantidad de publicaciones'] * 100

# Reemplazamos los NaN por ceros
df_prop_dolar.fillna(0, inplace=True)
# Redondeamos a 2 decimales
df_prop_dolar['Proporcion de publicaciones en dolares'] = df_prop_dolar['Proporcion de publicaciones en dolares'].round(2)

df_prop_dolar

In [ ]:
# Mergeamos nuevamente los datos geográficos de comuna con el df de proporción en dólares con key = comuna
comunas_prop_dolares = comunas.merge(df_prop_dolar, left_on= 'comuna', right_on= 'Comuna')

comunas_prop_dolares

In [ ]:
# Armamos el ùltimo gráfico de % de publicaciones en dólares por comuna
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
comunas_prop_dolares.plot(column="Proporcion de publicaciones en dolares",
             ax=ax,
             cmap="Oranges",
             edgecolor="black")

for idx, row in comunas_prop_dolares.iterrows():
    centroid = row.geometry.centroid
    ax.text(centroid.x, centroid.y,
            f'{row["Proporcion de publicaciones en dolares"]}',
            fontsize=8,
            ha='center',
            color='black')

plt.xticks([])
plt.yticks([])
plt.show()